In [3]:
import os
import io
import shutil

import mne

import numpy as np



In [4]:
filename = "dane_eeg.zip"
extract_dir = "dane_eeg"
seizure_occ_file = "dane_eeg/czasy_napadow.m"
healthy_dir = "dane_eeg/zdrowi"
diagnosed_dir = "dane_eeg/chorzy"

# shutil.unpack_archive(filename, extract_dir)

In [5]:

def get_mat_files(base_path): 
    mat_files_list = []
    filenames = {}

    for file in os.listdir(base_path):
        if file.endswith(".mat"):
            mat_files_list.append(base_path + "/" + file)
            
    for filename in mat_files_list:        
        filenames[filename.split('/')[-1].split('_')[0]] = filename
        
    return mat_files_list, filenames


healthy_paths, healthy_patients_dict = get_mat_files(healthy_dir)
diagnosed_paths, diagnosed_patients_dict = get_mat_files(diagnosed_dir)

In [6]:
import re

def get_all_occurrences(file_path):
    all_occurrences = {}
    with open (file_path , 'r') as file:
        for line in file :
            line_number = int(re.search(r'{([0-9]+)}', line).group(1))
            occ_list = list(map(int, re.search(r'\[(.+?)]', line).group(1).split()))
            all_occurrences[line_number] = occ_list
    return all_occurrences


occurences = get_all_occurrences(seizure_occ_file)

In [7]:
class Seizure:
    def __init__ (self, id, filename, samples, seizure_occurences=None, frequency=100, channels=16) -> None :
        super (). __init__ ()
        self.id = id
        self.filename = filename
        self.samples = samples
        self.seizure_occurences = seizure_occurences
        self.frequency = frequency
        self.channels = channels

In [8]:
def healthy_patients(healthy_people_dict):
    healthy_patients = []
    for id, path in healthy_people_dict.items():
        samples = io.loadmat(path)['x']
        filename = os.path.basename(path)
        healthy_people.append(Seizure(id, filename, samples))
 
    return healthy_patients

def unhealthy_patients(unhealthy_people_dict, seizure_occurences):
    unhealthy_patients = []
    for id, path in unhealthy_people_dict.items():
        seizure_occ = seizure_occurences[id]
        samples = io.loadmat(path)['x']
        filename = os.path.basename(path)
        unhealthy_people.append(Seizure(id, filename, samples, seizure_occurences=seizure_occ))
 
    return unhealthy_patients